# **Seminario Statistica Superiore: codice**

##**Figure Seminario**


Per il codice in questa sezione ho preso spunto da: 
1. https://github.com/ageron/handson-ml2/blob/master/04_training_linear_models.ipynb (per i grafici su gd-sgd-mb)
2. https://bayes-logistic.readthedocs.io/en/latest/usage.html (per capire la libreria bayes_logistic)
3. https://github.com/probml/pyprobml/tree/master/scripts (per i grafici su bayes logistic regression)

In [ ]:
pip install bayes_logistic

In [ ]:
pip install superimport

In [ ]:
#importo le librerie utilizzate
import pandas as pd
import os
import numpy as np
from sklearn import preprocessing
from sklearn import metrics
from sklearn.utils import shuffle
import matplotlib.pyplot as plt 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
import seaborn as sns


import sys
assert sys.version_info >= (3, 5)

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers
from keras import optimizers


import superimport
from scipy.stats import norm, multivariate_normal
from scipy.special import expit
import bayes_logistic 
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)



#per i plot
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

###Esempio ad hoc per confrontare gd-sgd-mb 




In [ ]:
#genero i dati
np.random.seed(40)
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)
X_b = np.c_[np.ones((100, 1)), X]
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
X_new = np.array([[0], [2]])
X_new_b = np.c_[np.ones((2, 1)), X_new]
y_predict = X_new_b.dot(theta_best)


In [ ]:
plt.plot(X_new, y_predict, "r-")
plt.plot(X, y, "b.")
plt.axis([0, 2, 0, 15])
plt.show()

##Gradient descent

In [ ]:
#implemento solo il caso di linear regression, più semplice
eta = 0.1  # learning rate
n_iterations = 1000
m = 100

theta_path_bgd = []
theta = np.random.randn(2,1) 

def gradient_descent_path(theta, eta, theta_path=None):
    m = len(X_b)
    n_iterations = 1000
    for iteration in range(n_iterations):
        if iteration < 10:
            y_predict = X_new_b.dot(theta)
        gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
        theta = theta - eta * gradients
        if theta_path is not None:
            theta_path.append(theta)


gradient_descent_path(theta, eta=0.1, theta_path=theta_path_bgd)

###Stochastic Gradient Descent

In [ ]:
theta_path_sgd = []
m = len(X_b)
np.random.seed(40)

In [ ]:
n_epochs = 50
t0, t1 = 5, 50  

def learning_schedule(t):
    return t0 / (t + t1)

theta = np.random.randn(2,1)  

for epoch in range(n_epochs):
    for i in range(m):
        if epoch == 0 and i < 20:                    
            y_predict = X_new_b.dot(theta)          
            style = "b-" if i > 0 else "r--"        
            plt.plot(X_new, y_predict, style)       
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients
        theta_path_sgd.append(theta)                

plt.plot(X, y, "b.")                                
plt.xlabel("$x_1$", fontsize=18)                    
plt.ylabel("$y$", rotation=0, fontsize=18)          
plt.axis([0, 2, 0, 15])                             
plt.savefig("sgd_plot")                                
plt.show()                                          

###Mini-batch gradient descent

In [ ]:
theta_path_mgd = []

n_iterations = 50
minibatch_size = 20

np.random.seed(40)
theta = np.random.randn(2,1)  # random initialization

t0, t1 = 200, 1000
def learning_schedule(t):
    return t0 / (t + t1)

t = 0
for epoch in range(n_iterations):
    shuffled_indices = np.random.permutation(m)
    X_b_shuffled = X_b[shuffled_indices]
    y_shuffled = y[shuffled_indices]
    for i in range(0, m, minibatch_size):
        t += 1
        xi = X_b_shuffled[i:i+minibatch_size]
        yi = y_shuffled[i:i+minibatch_size]
        gradients = 2/minibatch_size * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(t)
        theta = theta - eta * gradients
        theta_path_mgd.append(theta)

###Confronto tra i learning algorithm

In [ ]:
#trasformo in np.array
theta_path_bgd = np.array(theta_path_bgd)
theta_path_sgd = np.array(theta_path_sgd)
theta_path_mgd = np.array(theta_path_mgd)

In [ ]:
plt.figure(figsize=(7,4))
plt.plot(theta_path_sgd[:, 0], theta_path_sgd[:, 1], "r-s", linewidth=1, label="Stochastic")
plt.plot(theta_path_mgd[:, 0], theta_path_mgd[:, 1], "g-+", linewidth=2, label="Mini-batch")
plt.plot(theta_path_bgd[:, 0], theta_path_bgd[:, 1], "b-o", linewidth=3, label="Batch")
plt.legend(loc="upper left", fontsize=16)
plt.xlabel(r"$\theta_0$", fontsize=20)
plt.ylabel(r"$\theta_1$   ", fontsize=20, rotation=0)
plt.axis([2.5, 4.5, 2.3, 3.9])
plt.savefig("gradient_descent_paths_plot")
plt.show()

###Bayesian logistic regression

In [ ]:
np.random.seed(42)
#creo i dati
N = 30
D = 2
mu1 = np.hstack((np.ones((N,1)), 5 * np.ones((N, 1))))
mu2 = np.hstack((-5 * np.ones((N,1)), np.ones((N, 1))))
class1_std = 1
class2_std = 1.1
X_1 = np.add(class1_std*np.random.randn(N,2), mu1)
X_2 = np.add(2*class2_std*np.random.randn(N,2), mu2)
X = np.vstack((X_1,X_2))
t = np.vstack((np.ones((N,1)),np.zeros((N,1))))

#plotto i dati
x_1, y_1 = X[np.where(t==1)[0]].T
x_2, y_2 = X[np.where(t==0)[0]].T
plt.figure(0)
plt.scatter(x_1,y_1,c = 'red', s=20, marker = 'o')
plt.scatter(x_2,y_2,c = 'blue', s = 20, marker = 'o')

#plotto le ltu
alpha = 100
Range = 8
step = 0.1
xx, yy = np.meshgrid(np.arange(-Range,Range,step),np.arange(-Range,Range,step))
[n,n] = xx.shape
W = np.hstack((xx.reshape((n*n, 1)),yy.reshape((n*n, 1))))
Xgrid = W
ws = np.array([[3, 1], [4, 2], [5, 3], [7, 3]])
col = ['black', 'red', 'green', 'blue']
for ii in range(ws.shape[0]):
    w = ws[ii][:]
    pred = 1.0/(1+np.exp(np.dot(-Xgrid,w)))
    plt.contour(xx, yy, pred.reshape((n, n)), 1, colors=col[ii])
plt.title("dati")
plt.savefig('bayesian_1_.png')

In [ ]:
#log-likelihood
Xt = np.transpose(X)
f=np.dot(W,Xt)
log_prior = np.log(multivariate_normal.pdf(W, cov=(np.identity(D))*alpha))

log_like = np.dot(f, t) - np.sum(np.log(1+np.exp(f)), 1).reshape((n*n,1))
log_joint = log_like.reshape((n*n,1)) + log_prior.reshape((n*n,1))


plt.contour(xx, yy, -1*log_like.reshape((n,n)), 30)
plt.title("Log-Likelihood")

#plotto  i punti corrispondenti agli iperpiani separatori
for ii in range(0, ws.shape[0]):
    w = np.transpose(ws[ii, :])
    plt.annotate(str(ii+1), xy=(w[0], w[1]), color=col[ii])

j=np.argmax(log_like)
wmle = W[j, :]
slope = wmle[1] / wmle[0]

plt.plot([0, 7.9], [0, 7.9*slope])
plt.grid()
plt.savefig('bayesian_2_.png')

In [ ]:
#log distrib a posteriori non normalizzata
plt.contour(xx,yy,-1*log_joint.reshape((n,n)), 30)
plt.title("Log distribuz. a posteriori non normalizzata")
j2=np.argmax(log_joint)
wb = W[j2][:]
plt.scatter(wb[0], wb[1], c='red' , s = 100)
plt.grid()
plt.savefig('bayesian_3_.png')

In [ ]:
#approssimazione di laplace
#https://bayes-logistic.readthedocs.io/en/latest/usage.html
wfit, hfit = bayes_logistic.fit_bayes_logistic(t.reshape((N*D)), X, np.zeros(D), ((np.identity(D))*1/alpha), weights=None, solver='Newton-CG', bounds=None, maxiter=100)
co = np.linalg.inv(hfit)

log_laplace_posterior = np.log(multivariate_normal.pdf(W, mean = wfit, cov=co))
plt.contour(xx, yy, -1*log_laplace_posterior.reshape((n,n)), 30)
plt.scatter(wb[0], wb[1], c='red' , s = 100)
plt.title("Approssimazione di Laplace alla distr. a posteriori")
plt.grid()
plt.savefig('bayesian_4_.png')

In [ ]:
#p(y = 1 | x, wMAP)
pred = 1.0/(1+np.exp(np.dot(-Xgrid,wfit)))
plt.contour(xx, yy, pred.reshape((n,n)), 30)
x_1, y_1 = X[np.where(t == 1)[0]].T
x_2, y_2 = X[np.where(t == 0)[0]].T
plt.scatter(x_1, y_1, c='red', s=20, marker='o')
plt.scatter(x_2, y_2, c = 'blue', s=40, marker = 'o')
plt.title("p(y=1|x, wMAP)")
plt.savefig('approx_1_.png')

In [ ]:
#Decision boundary per w campionati
plt.scatter(x_1, y_1, c='red', s=20, marker='o')
plt.scatter(x_2, y_2, c='blue', s=20, marker='o')
predm = np.zeros((n*n,1))
s = 100
for i in range(s):
    wsamp = np.random.multivariate_normal(mean = wfit, cov=co)
    pred = 1.0/(1+np.exp(np.dot(-Xgrid,wsamp)))
    predm = np.add(predm, pred.reshape((n*n, 1)))
    plt.contour(xx, yy, pred.reshape((n,n)), np.array([0.5]))
plt.title("Decision boundary per w campionati dalla distr. pred. a posteriori")
plt.savefig('approx_2_.png')

In [ ]:
#MC
predm = predm/s
plt.contour(xx, yy, predm.reshape((n,n)), 30)
plt.scatter(x_1, y_1, c='red', s=20, marker='o')
plt.scatter(x_2, y_2, c='blue', s=20, marker='o')
plt.title("MC appross. di p(y=1|x)")
plt.savefig('approx_3_.png')

In [ ]:
#Approccio deterministico
plt.scatter(x_1, y_1, c='red', s=20, marker='o')
plt.scatter(x_2, y_2, c='blue', s=20, marker='o')
pr = bayes_logistic.bayes_logistic_prob(Xgrid, wfit, hfit)
plt.contour(xx, yy, pr.reshape((n, n)), 30)
plt.title("Proibit approximation")
plt.savefig('approx_4_.png')
plt.show()


### Sigmoid vs Proibit

In [ ]:
x = np.arange(-6, 6, 0.1)
l = np.sqrt(np.pi/8); 
plt.plot(x, expit(x), 'r-', label='sigmoid')
plt.plot(x, norm.cdf(l*x), 'g--', label='probit')

plt.axis([-6, 6, 0, 1])
plt.legend()
plt.savefig('sigmoid.png')
plt.show()


##**Problema dello spam**

In questa sezione il codice è tutta farina del mio sacco.

Ovviamente mi sono aiutato con le librerie e la relativa documentazione 

### Preprocessing

In [ ]:
#carico i dati
names = [1 + x for x in range(58)]
names[57] = 'target'
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data',
                   names = names)
data_x, data_y = data.drop('target', axis = 1), data['target']

In [ ]:
#vedo le prime osservazioni per farmi un'idea dei dati
data.head()

In [ ]:
#vedo la proporzione tra 0 e 1 e osservo che c'è bisogno di stratification
data['target'].value_counts()
sns.countplot(x='target', data=data, palette='hls')
plt.savefig('stratification')
plt.show()

In [ ]:
#divido in training e test dopo aver fatto shuffling
X, y = shuffle(data_x, data_y)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y)
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

###Model selection

In [ ]:
#definisco l'architettura
def create_model_sgd(learning_rate, momentum, k_reg):
    model = Sequential()
    model.add(Dense(1,
                    input_dim=57,
                    activation='sigmoid',
                    kernel_initializer= "uniform",
                    kernel_regularizer=regularizers.l2(k_reg)))
    optimizer = tf.keras.optimizers.SGD(lr=learning_rate, momentum=momentum)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
#prima grid search
np.random.seed(42)
model = KerasClassifier(build_fn=create_model_sgd, epochs = 150, batch_size = 40)
param_grid ={"learning_rate": [0.01, 0.05, 0.1],
             "momentum": [0.1, 0.2, 0.3],
             "k_reg": [0.00001, 0.0001, 0.001]}
model_cv = GridSearchCV(model,
                        param_grid,
                        scoring = None,
                        n_jobs=-1,
                        cv = 3,
                        verbose = 10)
model_result = model_cv.fit(X_train, y_train)

In [ ]:
#vedo i migliori parametri
model_cv.best_params_

In [ ]:
means = model_result.cv_results_['mean_test_score']
stds = model_result.cv_results_['std_test_score']
params = model_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) con: %r" % (mean, stdev, param))

In [ ]:
#risultato grid search
{'k_reg': 0.0001, 'learning_rate': 0.05, 'momentum': 0.3}

### Model assessment

In [23]:
def final_model_sgd():
    model = Sequential()
    model.add(Dense(1, input_dim=57, activation='sigmoid', kernel_initializer= "uniform", kernel_regularizer=regularizers.l2(0.0001)))
    optimizer = tf.keras.optimizers.SGD(lr=0.05, momentum=0.3)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
true_model = KerasClassifier(build_fn=final_model_sgd)
history = true_model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=150,
                    batch_size=40)

In [ ]:
#learning curve
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'test'], loc='upper left')
plt.savefig('learning_curve')
plt.show()